<a href="https://colab.research.google.com/github/BRIJNANDA1979/CNN-Sentinel/blob/master/CNN_Edusat_Data_8_classes_15_May_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu May 13 03:02:04 2021

@author: BRIJB
"""

from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import the needed packages
import tensorflow as tf
from keras import losses 
from keras import optimizers 
from keras import metrics 
import matplotlib.pyplot as plt
import matplotlib.image as img
import tensorflow.keras as keras
from keras.preprocessing import image
import numpy as np
from keras.models import model_from_json
import os

batch_size = 30

# define and move to dataset directory
datasetdir = "/content/drive/MyDrive/Data/Sentinel 2 Data for CNN/EuroSAT/2750"
import os
os.chdir(datasetdir)



# shortcut to the ImageDataGenerator class
ImageDataGenerator = keras.preprocessing.image.ImageDataGenerator
gen = ImageDataGenerator()
iterator = gen.flow_from_directory(
    os.getcwd(), 
    target_size=(64,64), 
    classes=('Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential','SeaLake')
    
)
# we can guess that the iterator has a next function, 
# because all python iterators have one. 
batch = iterator.next()
print(len(batch))
print(type(batch[0]))
print(batch[0].shape)
print(batch[0].dtype)
#print(batch[0].max())
#print(batch[1].shape)
#print(batch[1].dtype)
#print(type(batch[1]))
#the first element is an array of 32 images with 64X64 pixels, and 3 color channels, encoded as floats in the range 0 to 255
#The second element contains the 32 corresponding labels.


Found 16579 images belonging to 8 classes.
2
<class 'numpy.ndarray'>
(32, 64, 64, 3)
float32


In [2]:
batch = iterator.next()
print(len(batch))
print(type(batch[1]))
print(batch[1].shape)
print(batch[1].dtype)

2
<class 'numpy.ndarray'>
(32, 8)
float32


In [3]:
imgdatagen = ImageDataGenerator(
    rescale = 1/255., 
    validation_split = 0.2,
)

batch_size = 30
height, width = (64,64)
train_dataset = imgdatagen.flow_from_directory(
    os.getcwd(),
    target_size = (height, width), 
    classes = ('Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential','SeaLake'),
    batch_size = batch_size,
    subset = 'training'
)

val_dataset = imgdatagen.flow_from_directory(
    os.getcwd(),
    target_size = (height, width), 
    classes = ('Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential','SeaLake'),
    batch_size = batch_size,
    subset = 'validation'
)
#In this case out of 5000 images: Training: 4000 Validation: 20% of 4000 i.e. 1000
model = keras.models.Sequential()

Found 13264 images belonging to 8 classes.
Found 3315 images belonging to 8 classes.


In [4]:

initializers = {
    
}

model.add( 
    keras.layers.Conv2D(
        24, 5, input_shape=(64,64,3), 
        activation='relu', 
    )
)
model.add( keras.layers.MaxPooling2D(2) )
model.add( 
    keras.layers.Conv2D(
        48, 5, activation='relu', 
    )
)
model.add( keras.layers.MaxPooling2D(2) )
model.add( 
    keras.layers.Conv2D(
        96, 5, activation='relu', 
    )
)
model.add( keras.layers.Flatten() )
model.add( keras.layers.Dropout(0.4) )

model.add( keras.layers.Dense(
    8, activation='softmax',
    )
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 60, 60, 24)        1824      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 30, 24)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 48)        28848     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 48)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 96)          115296    
_________________________________________________________________
flatten (Flatten)            (None, 7776)              0         
_________________________________________________________________
dropout (Dropout)            (None, 7776)              0

In [5]:
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adamax(lr=0.001),
              metrics=['acc'])

In [6]:
history = model.fit_generator(
    train_dataset, 
    validation_data = val_dataset,
    workers=10,
    epochs=1,
)

#result of this model is not impressive, but can be imroved 
#443/443 [==============================] - 363s 802ms/step - loss: 0.3290 - acc: 0.3072 - val_loss: 0.2291 - val_acc: 0.5466

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


443/443 [==============================] - 363s 802ms/step - loss: 0.3290 - acc: 0.3072 - val_loss: 0.2291 - val_acc: 0.5466


In [7]:
import glob
import numpy as np
from PIL import Image
# https://stackoverflow.com/questions/37747021/create-numpy-array-of-images
X_data = []
files = glob.glob(r"/content/drive/MyDrive/Data/data/*.jpg")
for my_file in files:
    print(my_file)
    
    #image = Image.open(my_file).convert('RGB')
    image = Image.open(my_file)
    image = np.array(image)
   
    X_data.append(image)
X_data = np.array(X_data)
x = np.array(X_data[0])
print('X_data shape:', np.array(X_data).shape)
# If you have 20 64x64 images, this would print 'X_data shape: (20, 64,64,3)' 
# @tf.function(experimental_relax_shapes=True)
# def predict(x):
#      return model.predict(x)
output = model.predict(X_data)
print(output)

/content/drive/MyDrive/Data/data/Residential_1.jpg
/content/drive/MyDrive/Data/data/Pasture_2.jpg
/content/drive/MyDrive/Data/data/Pasture_1.jpg
/content/drive/MyDrive/Data/data/Forest_2.jpg
/content/drive/MyDrive/Data/data/Forest_5.jpg
/content/drive/MyDrive/Data/data/Forest_4.jpg
/content/drive/MyDrive/Data/data/Forest_3.jpg
/content/drive/MyDrive/Data/data/Forest_7.jpg
/content/drive/MyDrive/Data/data/Forest_9.jpg
/content/drive/MyDrive/Data/data/Forest_10.jpg
/content/drive/MyDrive/Data/data/Forest_8.jpg
/content/drive/MyDrive/Data/data/Forest_6.jpg
/content/drive/MyDrive/Data/data/Forest_1.jpg
X_data shape: (13, 64, 64, 3)
[[0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 1.6029401e-16 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 0.0000000e+00 1.3415359e-16 0.0000000e+00 0.0000000e+00
  0.0000000e+00 1.0000000e+00 0.0000000e+00]
 [0.0000000e+

In [8]:
# define and move to dataset directory
datasetdir = "/content/drive/MyDrive/Data/Sentinel 2 Data for CNN/EuroSAT"
import os
os.chdir(datasetdir)
model_1_EUROSAT_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_1_EUROSAT_json)
# serialize weights to HDF5
model.save_weights("model_1_EUROSAT_json.h5")
print("Saved model to disk")

Saved model to disk
